In [1]:
#importing and downloading revelant modules from NLTK
import nltk
nltk.download('vader_lexicon')
nltk.download('punkt')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/hima950/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /home/hima950/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import json
import os
import time
import yaml
os.chdir('../')
from api.couch_db import TweetsDB
from glob import glob

In [3]:
#open the config file for Db connections and folder paths
with open("config.yaml", 'r') as ymlfile:
    cfg = yaml.load(ymlfile)

In [4]:
#db connections
couch_db_conf = cfg['COUCHDB']
#create an instance of TWEETSDB and pass the connection details
couch_db = TweetsDB(couch_db_conf)
#path to senti_folder
senti_tasks_path= cfg['QUEUES']['sentiment_tasks']

In [5]:


def senti_analyse(input_doc):
    senti = SentimentIntensityAnalyzer()
    points = senti.polarity_scores(input_doc)
    if points['compound']<0.0:
        points['sentiment']= "Negative"
        if points['compound']<= -0.5:
            points['Intensity']="Strong"
        if -0.5 <points['compound']< 0.0:
            points['Intensity']="Moderate"
    if points['compound']>0.0:
        points['sentiment']= "Positive"
        if points['compound']>= 0.5:
            points['Intensity']="Strong"
        if 0.0<points['compound']<0.5:
            points['Intensity']="Moderate"
    if points['neu']==1:
        points['sentiment']="Neutral"       

    return points
            
    
    
    
  

In [ ]:
senti_analyse("me_shivaum @Miriam_McGuirk @ChrisJ_Pollard @NevilleGaunt @sagerbob @keswanger @DemarcioWash @BruceMctague… https://t.co/hSEOWBv5JC")

{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'sentiment': 'Neutral'}

In [ ]:
i = 1
while True:
    senti_tweets = glob('{}/*.txt'.format(senti_tasks_path))
 
    for path in senti_tweets:
        try:
            tweet_id = path.split('/')[-1].split('.')[0]
            with open(path, 'r') as fp:
                senti = fp.read()
            senti_doc = senti_analyse(senti)
            couch_db.update_document(tweet_id,senti_doc)
            try:
                os.remove(path)
            except Exception as e:
                           # if failed, report it back to the user 
                print("Error: {} .".format(e))
        except Exception as e:
            print('Tweet {} wasn\'t Senti Analysed and updated on DB due to error. {}'.format(tweet_id, e))
         
    print('Iteration: {}\tFiles processed: {}'.format(i, len(senti_tweets)))
    i+=1
    time.sleep(18)